<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkML0101ENSkillsNetwork20718538-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>

<h1 align="center"><font size="5">Classification with Python</font></h1>


<h2>Table of Contents</h2>
<div class="alert alert-block alert-info" style="margin-top: 20px">
    <ul>
        <li><a href="https://#Section_1"> Importing Data </a></li>
    <li><a href="https://#Section_2">Data Preprocessing</a> </li>
    <li><a href="https://#Section_3">One Hot Encoding </a></li>
    <li><a href="https://#Section_4">Training Data and Test Data </a></li>
    <li><a href="https://#Section_5">Instructions</a></li>
    <li><a href="https://#Section_6">Q1) Train a logistic regression model, identify best value of C parameter using GridSearchCV. Determine the accuracy of the test data. Also return Jaccard Index, log_loss and F1 Score on the test data
</a></li>
    <li><a href="https://#Section_7">Q2) Train a KNN model, find optimal values for the parameters : n_neighbors = [1,3,5,7,9], algorithm, and p, using GridSearchCV. Determine the accuracy of the test data . Also return Jaccard Index and F1 Score on the test data  </a></li>
    <li><a href="https://#Section_8">Q3) Train a SVM model,  find optimal values for parameters such as C = [.001, .01, .1, 1, 10, 100] and kernel=['linear', 'poly', 'rbf', 'sigmoid'],using GridSearchCV. Determine the accuracy of the test data . Also return Jaccard Index and F1 Score on the test data</a></li>
    <li><a href="https://#Section_9"> Q4) Train a Decision Tree, find optimal values for parameters such as criterion=['gini', 'entropy'], using GridSearchCV. Determine the accuracy of the test data . Also return Jaccard Index and F1 Score on the test data</a></li>
    <li><a href="https://#Section_10">Q5) Show the Accuracy,Jaccard,F1-Score and Log Loss in a tabular format using data frame for all of the above model</a></li>
<p>Estimated Time Needed: <strong>180 min</strong></p>
</div>

<hr>


In this notebook we try to practice all the classification algorithms that we learned in this course.

We load a dataset using Pandas library, and apply the following algorithms, and find the best one for this specific dataset by accuracy evaluation methods.

Lets first load required libraries:


In [1]:
#you are running the lab in your  browser, so we will install the libraries using ``piplite``
import piplite
await piplite.install(['pandas'])
await piplite.install(['matplotlib'])
await piplite.install(['numpy'])
await piplite.install(['seaborn'])
await piplite.install(['mock'])

In [32]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.metrics import jaccard_score
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score
from unittest.mock import MagicMock

Since sklearn calculates jaccard index differently than what was taught in the course we will define our own function for jaccard index


In [3]:
def jaccard_index(predictions, true):
    if (len(predictions) == len(true)):
        intersect = 0;
        for x,y in zip(predictions, true):
            if (x == y):
                intersect += 1
        return intersect / (len(predictions) + len(true) - intersect)
    else:
        return -1

### Data


#### About the Data


The original source of the data is Australian Government's Bureau of Meteorology and the latest data can be gathered from [http://www.bom.gov.au/climate/dwo/](http://www.bom.gov.au/climate/dwo/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkML0101ENSkillsNetwork20718538-2022-01-01).

The dataset we will use has extra columns like RainToday and our target RainTomorrow which was gathered from Rattle at [https://bitbucket.org/kayontoga/rattle/src/master/data/weatherAUS.RData](https://bitbucket.org/kayontoga/rattle/src/master/data/weatherAUS.RData?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkML0101ENSkillsNetwork20718538-2022-01-01)


This dataset is observations of weather metrics for each day from 2008 to 2017. The **weatherAUS.csv** dataset includes following fields:

| Field         | Description                                           | Unit            | Type   |
| ------------- | ----------------------------------------------------- | --------------- | ------ |
| Date          | Date of the Observation in YYYY-MM-DD                 | Date            | object |
| Location      | Location of the Observation                           | Location        | object |
| MinTemp       | Minimum temperature                                   | Celsius         | float  |
| MaxTemp       | Maximum temperature                                   | Celsius         | float  |
| Rainfall      | Amount of rainfall                                    | Millimeters     | float  |
| Evaporation   | Amount of evaporation                                 | Millimeters     | float  |
| Sunshine      | Amount of bright sunshine                             | hours           | float  |
| WindGustDir   | Direction of the strongest gust                       | Compass Points  | object |
| WindGustSpeed | Speed of the strongest gust                           | Kilometers/Hour | object |
| WindDir9am    | Wind direction averaged of 10 minutes prior to 9am    | Compass Points  | object |
| WindDir3pm    | Wind direction averaged of 10 minutes prior to 3pm    | Compass Points  | object |
| WindSpeed9am  | Wind speed averaged of 10 minutes prior to 9am        | Kilometers/Hour | float  |
| WindSpeed3pm  | Wind speed averaged of 10 minutes prior to 3pm        | Kilometers/Hour | float  |
| Humidity9am   | Humidity at 9am                                       | Percent         | float  |
| Humidity3pm   | Humidity at 3pm                                       | Percent         | float  |
| Pressure9am   | Atmospheric pressure reduced to mean sea level at 9am | Hectopascal     | float  |
| Pressure3pm   | Atmospheric pressure reduced to mean sea level at 3pm | Hectopascal     | float  |
| Cloud9am      | Fraction of the sky obscured by cloud at 9am          | Eights          | float  |
| Cloud3pm      | Fraction of the sky obscured by cloud at 3pm          | Eights          | float  |
| Temp9am       | Temperature at 9am                                    | Celsius         | float  |
| Temp3pm       | Temperature at 3pm                                    | Celsius         | float  |
| RainToday     | If there was rain today                               | Yes/No          | object |
| RISK_MM       | Amount of rain tomorrow                               | Millimeters     | float  |
| RainTomorrow  | If there is rain tomorrow                             | Yes/No          | float  |

Column definitions were gathered from [http://www.bom.gov.au/climate/dwo/IDCJDW0000.shtml](http://www.bom.gov.au/climate/dwo/IDCJDW0000.shtml?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkML0101ENSkillsNetwork20718538-2022-01-01)


### Importing the Dataset


In [4]:
from pyodide.http import pyfetch

async def download(url, filename):
    response = await pyfetch(url)
    if response.status == 200:
        with open(filename, "wb") as f:
            f.write(await response.bytes())

In [5]:
path='https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML0101EN-SkillUp/labs/ML-FinalAssignment/Weather_Data.csv'

In [6]:
await download(path, "Weather_Data.csv")
filename ="Weather_Data.csv"

In [7]:
df = pd.read_csv("Weather_Data.csv")
df.head()

,Date,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2/1/2008,19.5,22.4,15.6,6.2,0.0,W,41,S,SSW,...,92,84,1017.6,1017.4,8,8,20.7,20.9,Yes,Yes
1,2/2/2008,19.5,25.6,6.0,3.4,2.7,W,41,W,E,...,83,73,1017.9,1016.4,7,7,22.4,24.8,Yes,Yes
2,2/3/2008,21.6,24.5,6.6,2.4,0.1,W,41,ESE,ESE,...,88,86,1016.7,1015.6,7,8,23.5,23.0,Yes,Yes
3,2/4/2008,20.2,22.8,18.8,2.2,0.0,W,41,NNE,E,...,83,90,1014.2,1011.8,8,8,21.4,20.9,Yes,Yes
4,2/5/2008,19.7,25.7,77.4,4.8,0.0,W,41,NNE,W,...,88,74,1008.3,1004.8,8,8,22.5,25.5,Yes,Yes


### Data Preprocessing


#### One Hot Encoding


We need to perform one hot encoding to convert categorical variables to binary variables


In [8]:
df_sydney_processed = pd.get_dummies(data=df, columns=['RainToday', 'WindGustDir', 'WindDir9am', 'WindDir3pm'])

Next we replace the values of the RainTomorrow column changing it from a categorical column to a binary column. We do not use the get_dummies method because we would end up with two columns for RainTomorrow and we do not want that because it is our target.


In [9]:
df_sydney_processed.replace(['No', 'Yes'], [0,1], inplace=True)

### Training Data and Test Data


First we turn all columns into a float type. We don't need to do this because the **StandardScalar()** method will convert object types to float but it will give us a warning message.


In [10]:
df_sydney_processed.drop('Date',axis=1,inplace=True)

In [11]:
df_sydney_processed = df_sydney_processed.astype(float)

In [12]:
features = df_sydney_processed.drop(columns='RainTomorrow', axis=1)
Y = df_sydney_processed['RainTomorrow']

In [13]:
x_train, x_test, y_train, y_test = train_test_split(features, Y, test_size=.2, random_state=1)

In [14]:
norm = preprocessing.StandardScaler()

In [15]:
x_train = norm.fit_transform(x_train)

x_test = norm.transform(x_test)

### Instructions


Below is where we are going to use the classification algorithms to create a model based on our training data and finally evaluate our testing data using evaluation metrics learned in the course

We will some of the algorithms taught in the course, specifically

1.  Logistic Regression
2.  KNN
3.  SVM
4.  Decision Trees

We will evaluate our models using

1.  Accuracy Score
2.  Jaccard Index
3.  F1-Score
4.  Log Loss

Note: Jaccard Index is calculated differently in Sci Kit Learn so I have defined a function at the top of the notebook for you to use, its input style is the same as Sci Kit Learn

As we know these algorithms have many parameters and to find the best ones we will use GridSearchCV

Demonstration of how to do this using a mock classification algorithm :

*   Create a python dictionary with the key being the name of the parameters and the value being a list of possible values

*   Create an object of the classification algorithm

*   Create a GridSearchCV object and place your classification object and parameters dictionary as parameters, also define your GridSearchCV cv parameter (Use cv = 4)

*   Use the fit method of the GridSearchCV algorithm to train our model using x_train and y_train that we create before

*   Store the best model in a variable provided

*   Predict the target variable using the x_test data we created above

*   Calculate and store the values for each metric in the provided variables using the predictions and y_test data

You will need to research the parameters you need to use as there are many options but this is simple. GridSearchCV will determine the best model.

Finally using your models generate the report at the bottom


In [33]:

parameters = {'C': [.001, .01, .1, 1, 10, 100],
             'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
             'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']}

Mock = ProductionClass()

Grid = GridSearchCV(Mock, parameters, cv = 4)

Grid.fit(x_train, y_train)

BestMock = Grid.best_estimator_

predictions = BestMock.predict(x_test)

BestMock_Accuracy_Score = accuracy_score(predictions, y_test)
BestMock_JaccardIndex = jaccard_index(predictions, y_test)
BestMock_F1_Score = f1_score(predictions, y_test)
BestMock_Log_Loss = log_loss(y_test,BestLR.predict_proba(x_test)[:, 1])

<class 'NameError'>: name 'ProductionClass' is not defined

If you need some more help with grid search here are a couple of resourses

1.  [https://scikit-learn.org/stable/auto_examples/model_selection/plot_grid_search_digits.html](https://scikit-learn.org/stable/auto_examples/model_selection/plot_grid_search_digits.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkML0101ENSkillsNetwork20718538-2022-01-01)
2.  [https://scikit-learn.org/stable/modules/grid_search.html](https://scikit-learn.org/stable/modules/grid_search.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkML0101ENSkillsNetwork20718538-2022-01-01)


### Logistic Regression


For Logistic Regression please use the parameters C = \[.001, .01, .1, 1, 10, 100] and solver. Use the link provided to select the values for the solver parameter. [https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkML0101ENSkillsNetwork20718538-2022-01-01)

**Use max_iter parameter of LogisticRegression and set it as 1000**

Reference : LogisticRegression(max_iter=1000)


#### Q1) Train a Logistic Regression model, identify the best value of C parameter using GridSearchCV. Determine the accuracy of the test data. Also return Jaccard Index, LogLoss and F1-Score on the test data. Use cv parameter in GridSearchCV and assign it to 4


In [17]:
parameters = { }


In [18]:
#Enter Your Code Here and Execute

In [19]:
LR_Accuracy_Score = 
LR_JaccardIndex = 
LR_F1_Score = 
LR_Log_Loss = 

<class 'SyntaxError'>: invalid syntax (<ipython-input-19-54a47c5fa727>, line 1)

### KNN


#### Q2) Train a KNN model, find optimal values for the parameters : n_neighbors = \[1,3,5,7,9], algorithm, and p, using GridSearchCV. Determine the accuracy of the test data . Also return Jaccard Index and F1 Score on the test data.  Put cv = 4 in GridSearchCV()


In [20]:
parameters = {}

In [21]:
#Enter Your Code Here and Execute

In [22]:
KNN_Accuracy_Score = 
KNN_JaccardIndex = 
KNN_F1_Score = 

<class 'SyntaxError'>: invalid syntax (<ipython-input-22-80e78d7d019d>, line 1)

### SVM


#### Q3) Train a SVM model,  find optimal values for parameters such as C = \[.001, .01, .1, 1, 10, 100] and kernel=\['linear', 'poly', 'rbf', 'sigmoid'],using GridSearchCV. Determine the accuracy of the test data . Also return Jaccard Index and F1 Score on the test data.  Put cv = 4 in GridSearchCV()


In [23]:
parameters = {}

In [24]:
#Enter Your Code Here and Execute

In [25]:
SVM_Accuracy_Score = 
SVM_JaccardIndex = 
SVM_F1_Score = 

<class 'SyntaxError'>: invalid syntax (<ipython-input-25-7f39845ec73c>, line 1)

## Decision Tree


#### Q4) Train a Decision Tree, find optimal values for parameters such as criterion=\['gini', 'entropy'], using GridSearchCV. Determine the accuracy of the test data . Also return Jaccard Index and F1 Score on the test data.  Put cv = 4 in GridSearchCV()


In [26]:
parameters = {}

In [27]:
#Enter Your Code Here and Execute

In [28]:
Tree_Accuracy_Score = 
Tree_JaccardIndex = 
Tree_F1_Score = 

<class 'SyntaxError'>: invalid syntax (<ipython-input-28-23f6fc7f70df>, line 1)

### Report


#### Q5) Show the Accuracy,Jaccard Index,F1-Score and LogLoss in a tabular format using data frame for all of the above model

\*LogLoss is only for Logistic Regression Model


In [29]:
Report = pd.DataFrame({'Algorithm' : ['KNN', 'Decision Tree', 'SVM', 'LogisticRegression']})

Report['Accuracy'] = [LR_Accuracy_Score, KNN_Accuracy_Score, SVM_Accuracy_Score, Tree_Accuracy_Score]
Report['Jaccard'] = [LR_JaccardIndex, KNN_JaccardIndex, SVM_JaccardIndex, Tree_JaccardIndex]
Report['F1-Score'] = [LR_F1_Score, KNN_F1_Score, SVM_F1_Score, Tree_F1_Score]
Report['LogLoss'] = ['N/A', 'N/A', 'N/A', LR_Log_Loss]
Report

<class 'NameError'>: name 'LR_Accuracy_Score' is not defined

<h2 id="Section_5">  How to submit </h2>

#### **Make sure to toggle on 'Share with anyone who has the link' as shown below**

<p>Once you complete your notebook you will have to share it to be marked. Select the icon on the top right a marked in red in the image below, a dialogue box should open, select the option all&nbsp;content excluding sensitive code cells.</p>

<p><img height="440" width="700" src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML0101EN-SkillsNetwork/labs/FinalModule_SKO/images/share_notebook.png" alt="share notebook" /></p>
<p></p>

<p>You can then share the notebook&nbsp; via a&nbsp; URL by scrolling down as shown in the following image:</p>
<p style="text-align: center;"> <img height="308" width="350" src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML0101EN-SkillsNetwork/labs/FinalModule_SKO/images/url_notebook.png"  alt="share notebook" /> </p>


<h2>About the Authors:</h2> 

<a href="https://www.linkedin.com/in/joseph-s-50398b136/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkML0101ENSkillsNetwork20718538-2022-01-01">Joseph Santarcangelo</a> has a PhD in Electrical Engineering, his research focused on using machine learning, signal processing, and computer vision to determine how videos impact human cognition. Joseph has been working for IBM since he completed his PhD.

### Other Contributors

<a href="https://www.linkedin.com/in/birlahimanshu/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkML0101ENSkillsNetwork20718538-2022-01-01" target="_blank">Himanshu Birla</a>


## Change Log

| Date (YYYY-MM-DD) | Version | Changed By    | Change Description  |
| ----------------- | ------- | ------------- | ------------------- |
| 2020-08-27        | 1.0     | Malika Singla | Added lab to GitLab |

<hr>

## <h3 align="center"> © IBM Corporation 2020. All rights reserved. <h3/>
